In [4]:
import os
import json
import time
import azureml
import logging
from azureml.core.model import Model
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive import *
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.48


In [5]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Workspace configuration succeeded. You are all set!')
#except:
#    print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

# Compute Environment

In [11]:
cluster = 'gandalf'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC12', min_nodes=1, max_nodes=5)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)
    

Creating a new compute target...
Creating
Succeeded............
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


# Run Experiment

In [13]:
# Create and run experiment
mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='train.py',
                      use_gpu=True,
                      pip_requirements_file='requirements.txt')

run = mnist.submit(estimator)

In [14]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
mnist_lab,mnist_lab_1563567460_c085659c,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [15]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

# Manage Data

In [36]:
ds = ws.get_default_datastore()
mnist_data = ds.upload(src_dir = 'data', target_path = 'mnist', show_progress = True)

Uploading an estimated of 1 files
Target already exists. Skipping upload for mnist\mnist.npz
Uploaded 0 files


In [37]:
# run the same was as above
script_params={
    '--data': mnist_data.as_mount(),
}

# Create and run experiment
mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='train.py',
                      use_gpu=True,
                      pip_requirements_file='requirements.txt')

run = mnist.submit(estimator)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

# Optimize hyperparameters

In [ ]:
# same as above but increase the max_steps and remove the parameters
script_params={
    '--data': mnist_data,
    '--epochs': 100
}

mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='tfdigits.py',
                      use_gpu=True,
                      pip_requirements_file='requirements.txt')

## hyperparameter search

In [ ]:
ps = RandomParameterSampling(
    {
        '--layer': choice(128, 512, 1024)
        '--lr': loguniform(-15, -3),
        '--batch': choice(16, 32, 64, 128)
    }
)

early_termination_policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=2)

hyperdrive_run_config = HyperDriveRunConfig(estimator = estimator, 
                                            hyperparameter_sampling = ps, 
                                            policy = early_termination_policy,
                                            primary_metric_name = "accuracy",
                                            primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs = 10,
                                            max_concurrent_runs = 5)

hd_run = mnist.submit(hyperdrive_run_config)

RunDetails(hd_run).show()

In [8]:
exp = ws.experiments['simplemnist']
hd_run = [r for r in exp.get_runs() if r.id == 'simplemnist_1548818458668'][0]
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

# Automatic Machine Learning

In [ ]:
automl_settings = {
    "name": "AUTOMLMNIST_{0}".format(time.time()),
    "iteration_timeout_minutes": 5,
    "iterations": 20,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "preprocess": False,
    "max_concurrent_iterations": 10,
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(task='classification',
                             debug_log='automl_errors.log',
                             path='.',
                             compute_target = compute,
                             data_script='./get_data.py',
                             **automl_settings)

In [ ]:
experiment=Experiment(ws, 'mnist_lab')
remote_run = experiment.submit(automl_config)

In [9]:
exp = ws.experiments['mnist_lab']
remote_run = [r for r in exp.get_runs() if r.id == 'AutoML_b41af9f3-998c-4f3c-8493-9a076f702631'][0]
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

# Deploy best model

In [ ]:
hd_run = [r for r in exp.get_runs() if r.id == 'simplemnist_1548818458668'][0]
best = [r for r in hd_run.get_children() if r.id == 'simplemnist_1548818458668_3'][0]
best.get_file_names()

In [ ]:
best.download_file(name='outputs/digits.pb', output_file_path='outputs')
model_file = 'outputs/digits.pb'
model = Model.register(ws, model_name='SimpleMNIST', model_path=model_file, 
                       description='Simple MNIST model extracted from hyperparamter optimization run')

## Create Conda Deps

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
myenv = CondaDependencies()

myenv.add_pip_package('numpy')
myenv.add_tensorflow_pip_package()

with open('simplemnist.yml','w') as f:
    print('Writing out {}'.format('simplemnist.yml'))
    f.write(myenv.serialize_to_string())
    print('Done!')

## Create Image

In [ ]:
model = ws.models['mnist_lab']

In [ ]:
from azureml.core.image import ContainerImage, Image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                    runtime="python", 
                                    conda_file="simplemnist.yml")


image = Image.create(ws, 'mnist_lab', [model], image_config)
image.wait_for_creation(show_output=True)

In [ ]:
image.image_build_log_uri

# Deploy Service
Deploy newly created image!

## ACI Deployment

In [12]:
image = ws.images['simplemnist']
service_name = 'simplemnist-svc'

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

if service_name in ws.webservices:
    ws.webservices[service_name].delete()

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               description='simple MNIST digit detection')
service = Webservice.deploy_from_image(workspace=ws, 
                                       image=image, 
                                       name=service_name, 
                                       deployment_config=aciconfig)
service.wait_for_deployment(show_output=True)

In [ ]:
with open('deploy.log','w') as f:
    f.write(service.get_logs())
service.scoring_uri

## Enable App Insights

In [ ]:
service.update(enable_app_insights=True)

## Kubernetes Deployment

In [ ]:
from azureml.core.webservice import Webservice, AksWebservice

# get aks target
aks_target = ws.compute_targets['eagles']
# Set configuration and service name
aks_config = AksWebservice.deploy_configuration()
aks_service_name ='simplemnist-aks-svc'

if aks_service_name in ws.webservices:
    ws.webservices[aks_service_name].delete()
    
# Deploy from image
akssvc = Webservice.deploy_from_image(workspace = ws, 
                                            name = aks_service_name,
                                            image = image,
                                            deployment_config = aks_config,
                                            deployment_target = aks_target)
# Wait for the deployment to complete
akssvc.wait_for_deployment(show_output = True)
print(akssvc.state)

In [ ]:
akssvc.update(collect_model_data=True, enable_app_insights=True)